In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyam
import pymagicc
import scmdata
import silicone.multiple_infillers as mi
import sys

<IPython.core.display.Javascript object>

In [2]:
sys.path.insert(0, '..')

In [3]:
import scripts.utils

In [4]:
sr15 = pyam.IamDataFrame("..\input\complete_sr15_emissions.csv").filter(region="World")

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ..\input\complete_sr15_emissions.csv


In [5]:
rcmip = pyam.IamDataFrame("../input/rcmip-emissions-annual-means-v5-1-0.csv")

pyam.core - INFO: Reading file ..\input\rcmip-emissions-annual-means-v5-1-0.csv


In [6]:
rcmip.filter(scenario="ssp534-over").model

['REMIND-MAGPIE']

In [7]:
version = "v5"
outdir = f'../output/chosen_files/{version}/'
scen_file = outdir + "scen_{}"
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [8]:
med_temp = 'AR5 climate diagnostics|Temperature|Global Mean|MAGICC6|MED'
sr15.set_meta(meta='uncategorized', name='warming_category')
sr15.categorize(
    'warming_category', 'below 1.6C',
    criteria={med_temp: {'up': 1.6, 'lo': 1.5, 'year': 2100}},
    color='xkcd:blue'
)
sr15.categorize(
    'warming_category', 'below 1.5C',
    criteria={med_temp: {'up': 1.5, 'year': 2100}},
    color='xkcd:baby blue'
)

pyam.core - INFO: No scenarios satisfy the criteria
pyam.core - INFO: No scenarios satisfy the criteria


In [9]:
peak = 'peak_temperature'
sr15.set_meta_from_data(name=peak, variable=med_temp, method=np.max)

pyam.core - WARNING: Filtered IamDataFrame is empty!


In [10]:
sr15.filter(
    peak_temperature=[x for x in sr15.meta.peak_temperature if x < 2 and x > 1.8], scenario="SSP*"
)

pyam.core - WARNING: Filtered IamDataFrame is empty!


<class 'pyam.core.IamDataFrame'>
Index:
 * model    : (0)
 * scenario : (0)
Timeseries data coordinates:
   region   : (0)
   variable : (0)
   unit     : (0)
   year     : (0)
Meta indicators:
   exclude (bool) (0)
   warming_category (object) (0)
   peak_temperature (object) (0)

In [11]:
sr15.filter(scenario="SSP*").model

['AIM/CGE 2.0',
 'GCAM 4.2',
 'IMAGE 3.0.1',
 'MESSAGE-GLOBIOM 1.0',
 'REMIND-MAgPIE 1.5',
 'WITCH-GLOBIOM 3.1']

In [12]:
rcmip.scenario

['esm-bell-1000PgC',
 'esm-bell-2000PgC',
 'esm-bell-750PgC',
 'esm-pi-CO2pulse',
 'esm-pi-cdr-pulse',
 'esm-piControl',
 'historical',
 'historical-cmip5',
 'rcp26',
 'rcp45',
 'rcp60',
 'rcp85',
 'ssp119',
 'ssp126',
 'ssp245',
 'ssp370',
 'ssp370-lowNTCF-aerchemmip',
 'ssp370-lowNTCF-gidden',
 'ssp434',
 'ssp460',
 'ssp534-over',
 'ssp585']

In [13]:
rcmip.filter(scenario="ssp119")

<class 'pyam.core.IamDataFrame'>
Index:
 * model       : IMAGE (1)
 * scenario    : ssp119 (1)
Timeseries data coordinates:
   region      : World, World|R5.2ASIA, World|R5.2LAM, ... World|R5.2REF (6)
   variable    : Emissions|BC, ... (182)
   unit        : Mt BC/yr, Mt CH4/yr, Mt CO/yr, Mt CO2/yr, ... kt cC4F8/yr (51)
   year        : 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, ... 2500 (315)
   mip_era     : CMIP6 (1)
   activity_id : not_applicable (1)
Meta indicators:
   exclude (bool) False (1)

In [45]:
desired_scenarios = []
desired_scenarios.append(rcmip.filter(model="REMIND-MAGPIE", scenario="ssp534-over"))
desired_scenarios.append(rcmip.filter(model="IMAGE", scenario="ssp119"))

In [54]:
df = pyam.concat(desired_scenarios).filter(region="World", variable="Emissions*")

# Infill the missing emissions

In [55]:
df = df.data
del df["activity_id"]
del df["mip_era"]

In [56]:
df["variable"] = df["variable"].str.replace("MAGICC AFOLU", "AFOLU").replace(
    "MAGICC Fossil and Industrial", "Energy and Industrial Processes"
)

In [57]:
df = pyam.IamDataFrame(df)

In [59]:
missing_emissions = [
    'Emissions|PFC|C2F6', 'Emissions|HFC|HFC43-10', 'Emissions|PFC|C6F14', 'Emissions|HFC|HFC125', 
    'Emissions|HFC|HFC32', 'Emissions|HFC|HFC143a', 'Emissions|HFC|HFC245fa', 'Emissions|HFC|HFC23', 
    'Emissions|HFC|HFC134a', 'Emissions|SF6', 'Emissions|PFC|CF4', 'Emissions|HFC|HFC227ea', 
]
leader = "Emissions|CO2"
cent = [y for y in df.year if y<=2100 and y > 2000]
infillee_df = df.filter(year=cent, variable=leader)

In [60]:
for em in missing_emissions:
    is_empty = sr15.filter(region="World", variable=em).empty
    if is_empty:
        print(em)

In [61]:
infilled_df = mi.infill_all_required_variables(
    infillee_df.filter(year=cent), 
    sr15.filter(region="World", year=cent), [leader], 
    required_variables_list=missing_emissions
)

Filling required variables: 100%|██████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.88it/s]


In [62]:
df = infilled_df.append(df.filter(variable="Emissions|CO2", keep=False))

In [72]:
normal_years = [2015] + list(np.arange(2020, 2101, 10))
normal_years
bad_vars = [
    "*Sulfur|*", "*|BC|*", "*CH4|*", "Emissions|Montreal Gases|*", "Emissions|VOC|*", 
    "Emissions|NH3|*", "Emissions|NOx|*", "Emissions|OC|*", "Emissions|CO|*", 
]

In [73]:
for scenario in df.scenario:
    scripts.utils.construct_scen_file(
        df.filter(scenario=scenario, variable="Emissions|*", year=normal_years).filter(
            variable=bad_vars, keep=False
        ), 
        scen_file.format(scenario)
    )

pyam.utils - WARNING: Formatted data is empty!
pyam.utils - WARNING: Formatted data is empty!
